# Random Forest Regression to Forecast Merchant Monthly Revenue

In [1]:
# Initialise a spark session
import pandas as pd
from collections import Counter
import os
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


spark = (
    SparkSession.builder.appName("RF Model")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "8g")  # Increase driver memory
    .config("spark.executor.memory", "8g")  # Increase executor memory
    .config("spark.executor.instances", "4")  # Increase the number of executor instances
    .config("spark.driver.maxResultSize", "2g")
    .config("spark.sql.shuffle.partitions", "200") \
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/19 14:33:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Read transaction file
transactions = spark.read.parquet('../data/curated/flagged_fraud')
transactions = transactions.filter(F.col("is_fraud") != True)

In [3]:
transactions.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- year_week: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- fraud_probability_consumer: double (nullable = true)
 |-- name_consumer: string (nullable = true)
 |-- address_consumer: string (nullable = true)
 |-- state_consumer: string (nullable = true)
 |-- postcode_consumer: integer (nullable = true)
 |-- gender_consumer: string (nullable = true)
 |-- name_merchant: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- fraud_probability_merchant: double (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- order_month_year: string (nullable = true)
 |-- SA4_CODE_2011: string (nullable = true)
 |-- SA4_NAME_2011: string (nullable = true)
 |-- unemployment_rate: string (nullable = true)
 |-- z_score: double (nullable = true)
 |-- consumer_weekly_transaction: long (nullabl

### Feature Engineering

In [4]:
# Aggregating monthly revenue for each merchant
monthly_revenue_df = transactions.groupBy('merchant_abn', 'order_month_year', 'name_merchant').agg(
    F.sum('dollar_value').alias('monthly_revenue'),
    F.count('order_id').alias('transaction_count'),
    F.avg('fraud_probability_merchant').alias('avg_fraud_probability_merchant'),
    F.first('tags').alias('merchant_tags')  # Assuming tags are constant per merchant
)
    
# Aggregating consumer-level features (most common state and gender for each merchant)

# Most common consumer state per merchant
consumer_state_mode = transactions.groupBy('merchant_abn', 'state_consumer').count() \
    .withColumn('row_num', F.row_number().over(Window.partitionBy('merchant_abn').orderBy(F.desc('count')))) \
    .filter(F.col('row_num') == 1) \
    .select('merchant_abn', 'state_consumer')

# Most common consumer gender per merchant
consumer_gender_mode = transactions.groupBy('merchant_abn', 'gender_consumer').count() \
    .withColumn('row_num', F.row_number().over(Window.partitionBy('merchant_abn').orderBy(F.desc('count')))) \
    .filter(F.col('row_num') == 1) \
    .select('merchant_abn', 'gender_consumer')

# Average Unemployment Rate per Merchant Month-Year
transactions = transactions.withColumn("unemployment_rate_numeric", F.col("unemployment_rate").cast("float"))

unemployment_agg = transactions.groupBy('merchant_abn', 'order_month_year').agg(
    F.avg('unemployment_rate_numeric').alias('avg_unemployment_rate')
)

In [5]:
# Joining Datasets
monthly_revenue_df = monthly_revenue_df.join(consumer_state_mode, on='merchant_abn', how='left') \
                                      .join(consumer_gender_mode, on='merchant_abn', how='left')

# Join with unemployment data on both 'merchant_abn' and 'order_month_year'
monthly_revenue_df = monthly_revenue_df.join(unemployment_agg, on=['merchant_abn', 'order_month_year'], how='left')

# Show the final dataframe
monthly_revenue_df.show(5)

+------------+----------------+--------------------+------------------+-----------------+------------------------------+--------------------+--------------+---------------+---------------------+
|merchant_abn|order_month_year|       name_merchant|   monthly_revenue|transaction_count|avg_fraud_probability_merchant|       merchant_tags|state_consumer|gender_consumer|avg_unemployment_rate|
+------------+----------------+--------------------+------------------+-----------------+------------------------------+--------------------+--------------+---------------+---------------------+
| 10023283211|          Aug-21|       Felis Limited|15807.479921460477|               86|              56.2429773174978|((furniture, home...|           NSW|           Male|    80.79883797224178|
| 10023283211|          Mar-21|       Felis Limited| 9076.307821688919|               40|             56.40749878739966|((furniture, home...|           NSW|           Male|     78.1724992275238|
| 10023283211|          M

In [6]:
# Checking for Missing Values
nulls = monthly_revenue_df.agg(
    *(F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)).alias(c) for c in monthly_revenue_df.columns)
)

# Show the result
nulls.show()

24/09/19 14:34:06 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+------------+----------------+-------------+---------------+-----------------+------------------------------+-------------+--------------+---------------+---------------------+
|merchant_abn|order_month_year|name_merchant|monthly_revenue|transaction_count|avg_fraud_probability_merchant|merchant_tags|state_consumer|gender_consumer|avg_unemployment_rate|
+------------+----------------+-------------+---------------+-----------------+------------------------------+-------------+--------------+---------------+---------------------+
|           0|               0|            0|              0|                0|                             0|            0|             0|              0|                    0|
+------------+----------------+-------------+---------------+-----------------+------------------------------+-------------+--------------+---------------+---------------------+



In [7]:
# Creating lag features to include previous month's revenue
window_spec = Window.partitionBy('merchant_abn').orderBy('order_month_year')

# Lagging features: Previous month's revenue
monthly_revenue_df = monthly_revenue_df.withColumn(
    'previous_month_revenue', F.lag('monthly_revenue', 1).over(window_spec)
)

# Calculate revenue growth (percentage change)
monthly_revenue_df = monthly_revenue_df.withColumn(
    'revenue_growth',
    F.when(F.col('previous_month_revenue') > 0, 
           (F.col('monthly_revenue') - F.col('previous_month_revenue')) / F.col('previous_month_revenue'))
    .otherwise(F.lit(0))  # Fill with 0 if there is no previous revenue
)

# Fill NA values for first month with 0 (no previous data available)
monthly_revenue_df = monthly_revenue_df.fillna({'previous_month_revenue': 0, 'revenue_growth': 0})


monthly_revenue_df = monthly_revenue_df.fillna(0)  # Filling NA values for first month
monthly_revenue_df.show(5)

+------------+----------------+-------------+------------------+-----------------+------------------------------+--------------------+--------------+---------------+---------------------+----------------------+--------------------+
|merchant_abn|order_month_year|name_merchant|   monthly_revenue|transaction_count|avg_fraud_probability_merchant|       merchant_tags|state_consumer|gender_consumer|avg_unemployment_rate|previous_month_revenue|      revenue_growth|
+------------+----------------+-------------+------------------+-----------------+------------------------------+--------------------+--------------+---------------+---------------------+----------------------+--------------------+
| 10023283211|          Apr-21|Felis Limited|   9221.4058068711|               47|             56.03849374950703|((furniture, home...|           NSW|           Male|    74.54042625427246|                   0.0|                 0.0|
| 10023283211|          Aug-21|Felis Limited|15807.479921460477|        

### Data Preparation

In [8]:
from pyspark.ml.feature import StandardScaler, StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

# StringIndexing categorical columns (merchant_tags, consumer_state, gender_consumer)
indexers = [
    StringIndexer(inputCol='merchant_tags', outputCol='merchant_tags_indexed', handleInvalid='keep'),
    StringIndexer(inputCol='state_consumer', outputCol='state_consumer_indexed', handleInvalid='keep'),
    StringIndexer(inputCol='gender_consumer', outputCol='gender_consumer_indexed', handleInvalid='keep')
]

# OneHotEncoding indexed columns
encoders = [
    OneHotEncoder(inputCol='merchant_tags_indexed', outputCol='merchant_tags_encoded'),
    OneHotEncoder(inputCol='state_consumer_indexed', outputCol='state_consumer_encoded'),
    OneHotEncoder(inputCol='gender_consumer_indexed', outputCol='gender_consumer_encoded')
]

# VectorAssembler to combine numeric features into a single feature vector
assembler = VectorAssembler(
    inputCols=[
        'monthly_revenue', 'transaction_count', 'avg_fraud_probability_merchant', 'avg_unemployment_rate',
        'merchant_tags_encoded', 'state_consumer_encoded', 'gender_consumer_encoded', 'revenue_growth'
    ], 
    outputCol='features'
)

# Standardizing the numeric features
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')

pipeline = Pipeline(stages=indexers + encoders + [assembler, scaler])

# Fit the pipeline to the dataset
model_pipeline = pipeline.fit(monthly_revenue_df)

final_df = model_pipeline.transform(monthly_revenue_df)

final_df.select('merchant_abn', 'order_month_year', 'scaled_features').show(5)


24/09/19 14:34:26 WARN DAGScheduler: Broadcasting large task binary with size 57.4 MiB
24/09/19 14:34:32 WARN DAGScheduler: Broadcasting large task binary with size 57.4 MiB
24/09/19 14:34:41 WARN DAGScheduler: Broadcasting large task binary with size 23.3 MiB


+------------+----------------+--------------------+
|merchant_abn|order_month_year|     scaled_features|
+------------+----------------+--------------------+
| 10023283211|          Apr-21|(439966,[0,1,2,3,...|
| 10023283211|          Aug-21|(439966,[0,1,2,3,...|
| 10023283211|          Dec-21|(439966,[0,1,2,3,...|
| 10023283211|          Feb-22|(439966,[0,1,2,3,...|
| 10023283211|          Jan-22|(439966,[0,1,2,3,...|
+------------+----------------+--------------------+
only showing top 5 rows



### Training Model

In [9]:
train_data, test_data = final_df.randomSplit([0.8, 0.2], seed=42)

In [10]:
rf = RandomForestRegressor(featuresCol='scaled_features', labelCol='monthly_revenue', maxBins=50, maxDepth=15)
rf_model = rf.fit(train_data)

# Make predictions on the test data
predictions = rf_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol='monthly_revenue', predictionCol='prediction', metricName='rmse')

# Root Mean Squared Error (RMSE)
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# R-squared
r2_evaluator = RegressionEvaluator(labelCol='monthly_revenue', predictionCol='prediction', metricName='r2')
r2 = r2_evaluator.evaluate(predictions)
print(f"R-squared: {r2}")

predictions.select('merchant_abn', 'order_month_year', 'monthly_revenue', 'revenue_growth','prediction').show(5)

24/09/19 14:34:56 WARN DAGScheduler: Broadcasting large task binary with size 80.1 MiB
24/09/19 14:34:58 WARN DAGScheduler: Broadcasting large task binary with size 80.1 MiB
24/09/19 14:35:03 WARN DAGScheduler: Broadcasting large task binary with size 84.5 MiB
24/09/19 14:36:12 WARN DAGScheduler: Broadcasting large task binary with size 1728.3 KiB
24/09/19 14:36:13 WARN DAGScheduler: Broadcasting large task binary with size 96.9 MiB
24/09/19 14:36:18 WARN MemoryStore: Not enough space to cache rdd_443_3 in memory! (computed 457.9 MiB so far)
24/09/19 14:36:18 WARN MemoryStore: Not enough space to cache rdd_443_5 in memory! (computed 457.9 MiB so far)
24/09/19 14:36:18 WARN BlockManager: Persisting block rdd_443_3 to disk instead.
24/09/19 14:36:18 WARN BlockManager: Persisting block rdd_443_5 to disk instead.
24/09/19 14:36:18 WARN MemoryStore: Not enough space to cache rdd_443_6 in memory! (computed 457.9 MiB so far)
24/09/19 14:36:18 WARN BlockManager: Persisting block rdd_443_6 to d

[158.816s][warning][gc,alloc] Executor task launch worker for task 5.0 in stage 207.0 (TID 589): Retried waiting for GCLocker too often allocating 219985 words


24/09/19 14:36:30 WARN BlockManager: Block rdd_443_5 could not be removed as it was not found on disk or in memory
24/09/19 14:36:30 ERROR Executor: Exception in task 5.0 in stage 207.0 (TID 589)
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.ml.tree.impl.TreePoint$.labeledPointToTreePoint(TreePoint.scala:91)
	at org.apache.spark.ml.tree.impl.TreePoint$.$anonfun$convertToTreeRDD$4(TreePoint.scala:75)
	at org.apache.spark.ml.tree.impl.TreePoint$$$Lambda/0x000000d8023fa950.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.storage.memory.PartiallyUnrolledIterator.next(MemoryStore.scala:785)
	at org.apache.spark.serializer.SerializationStream.writeAll(Serializer.scala:140)
	at org.apache.spark.serializer.SerializerManager.dataSerializeStream(SerializerManager.scala:177)
	at org.apache.spark.storage.BlockManager.$anonfun$doPutIterator$3(BlockManager.scal

ConnectionRefusedError: [Errno 61] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/melissaputri/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/melissaputri/anaconda3/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/melissaputri/anaconda3/lib/python3.11/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


## Predicting Future Monthly Revenue

In [72]:
from pyspark.sql.types import DateType
from dateutil.relativedelta import relativedelta
from datetime import datetime

# Step 1: Parse the order_month_year column to a proper date format
monthly_revenue_df = monthly_revenue_df.withColumn(
    'order_month_year_date', F.to_date(F.concat(F.lit('01-'), F.col('order_month_year')), 'dd-MMM-yy')
)

# Get the most recent month per merchant
window_spec = Window.partitionBy('merchant_abn').orderBy(F.desc('order_month_year_date'))
latest_merchant_data = monthly_revenue_df.withColumn('row_num', F.row_number().over(window_spec)) \
                                         .filter(F.col('row_num') == 1) \
                                         .drop('row_num')

In [73]:
next_month = 'Aug-24'
future_month_df = spark.createDataFrame([(next_month,)], ['future_order_month_year'])
future_data = latest_merchant_data.crossJoin(future_month_df)

In [ ]:
future_data.show(5)

In [ ]:
future_data = model_pipeline.transform(future_data)
future_data = rf_model.transform(future_data)
future_predictions = future_data.select('merchant_abn', 'future_order_month_year', 'prediction')
future_predictions = future_predictions.withColumnRenamed('prediction', 'projected_revenue')
future_predictions.show(5)

In [ ]:
top_10_predictions = future_predictions.orderBy(F.col('projected_revenue').desc())

# Show the top 10 merchants by predicted revenue
top_10_predictions.select('merchant_abn', 'merchant_name', 'projected_revenue').show(10)